# RAPIDS and Cybersecurity : A Network Use Case

### KDD 2019 Hands-On Tutorial
### Authors
 - Rachel Allen, PhD (NVIDIA) [rachela@nvidia.com]
 - Haekyu Park (Georgia Tech, NVIDIA)
 - Bartley Richardson, PhD (NVIDIA) [brichardson@nvidia.com]

### Table of Contents
* Introduction
* Data Importing and Formatting
* Data Exploration
    * Dataset Summary
    * Dataset Size and Data Types
    * Summary Statistics
    * Value Counts
    * Aggregate Statistics
    * Visualizing Data Distributions
* Graph Representation 
    * Degree
    * PageRank
    * Personalized PageRank
    * Graph Visualization
    
### Development Notes
 - Developed using: RAPIDS v0.9.0
 - Last tested using: RAPIDS v0.9.0 on August 20, 2019

In [ ]:
import cudf
import pandas as pd
import dask_cudf
import cugraph

# for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Introduction

Today, organizations collect vast amounts of network traffic and network metadata. As the volume of data collected and velocity at which it's collected continue to increase, security analysts and forensic investigators require fast triage, processing, modeling, and visualization capabilities. Using the [RAPIDS](https://rapids.ai) suite of open-source software, we demonstrate how to:

1. Triage and perform data exploration,
2. Model network data as a graph,
3. Perform graph analytics on the graph representation of the cyber network data, and
4. Prepare the results in a way that is suitable for visualization.

# Data Import and Formatting

For this tutorial, we will used the [IDS 2018 dataset](https://www.unb.ca/cic/datasets/ids-2018.html) from the [Canadian Institute for Cybersecurity](https://www.unb.ca/cic/). The data is stored as raw PCAP files in AWS, and we'll need flow-level data for this use case. To make things eaiser, we've already created bidirectional flow using the [CIC FlowMeter tool](https://github.com/ISCX/CICFlowMeter). You can download it and get started immediately using the cell below. If you wish to store the data in a different location other than the default, change the value of `BASE_DIRECTORY`.

In [ ]:
BASE_DIRECTORY = "../../../../data/"
DOWNLOAD_DIRECTORY = BASE_DIRECTORY + "cic_ids2018/"
DOWNLOAD_FILE = "Friday-02-03-2018-biflows.tar.gz"
DIR_AND_FILE = DOWNLOAD_DIRECTORY + DOWNLOAD_FILE
DATA_DIRECTORY = DOWNLOAD_DIRECTORY + DOWNLOAD_FILE.split('.')[0] + "/"

!mkdir -p $DOWNLOAD_DIRECTORY
!if [ ! -f $DIR_AND_FILE ]; then echo ">> Downloading data" && wget -O $DIR_AND_FILE https://rapidsai-data.s3.us-east-2.amazonaws.com/cyber/kdd2019/Friday-02-03-2018-biflows.tar.gz; else echo ">> Data already downloaded"; fi
!if [ ! -d $DATA_DIRECTORY ]; then echo ">> Extracting $DOWNLOAD_FILE to $DATA_DIRECTORY" && tar -xzf $DIR_AND_FILE -C $DOWNLOAD_DIRECTORY; else echo ">> Data already extracted to $DATA_DIRECTORY"; fi

If you would prefer to create your own biflow data, [follow the directions at the bottom of this page to download the data](https://www.unb.ca/cic/datasets/ids-2018.html) to your machine. You'll then need to build and use the CIC FlowMeter tool to create biflow data.

Load in `conn` (connection) logs into a GDF (GPU Data Frame) using Dask-cuDF. Then we compute the GDF to convert to a cuDF data frame. Dask-cuDF is appropriate if your data doesn't fit in memory on a single GPU, you want to distribute your workflow across multiple GPUs, or (as is the case here) [you want to analyze data spread across many files at once](https://rapidsai.github.io/projects/cudf/en/latest/10min.html#When-to-use-cuDF-and-Dask-cuDF).

In [ ]:
# specify the location of the data
DATA_LOCATION = DATA_DIRECTORY + "*.csv"

In [ ]:
%%time
dask_gdf = dask_cudf.read_csv(DATA_LOCATION, 
                              sep=',',
                              dtype=["str","str","int32","str","int32","int32","str","int32","int32","int32",
                                     "float32","float32","float32","float32","float32","float32","float32",
                                     "float32","float32","float32","str","str","float32","float32","float32",
                                     "float32","float32","float32","float32","float32","float32","float32",
                                     "float32","float32","float32","float32","int32","int32","int32","int32",
                                     "int32","int32","float32","float32","float32","float32","float32","float32",
                                     "float32","int32","int32","int32","int32","int32","int32","int32","int32",
                                     "float32","float32","float32","float32","int32","int32","int32","int32",
                                     "int32","int32","int32","int32","int32","int32","int32","int32","int32",
                                     "int32","float32","float32","float32","float32","float32","float32","float32",
                                     "float32","str"])

In [ ]:
%%time
gdf = dask_gdf.compute()

We'll inspect the head of the new cuDF as a sanity check.

In [ ]:
gdf.head().to_pandas()

The timestamp is in string format, so we'll convert to epoch time.

In [ ]:
%%time
gdf['datetime'] = gdf['Timestamp'].str.timestamp2int(format='%d/%m/%Y %I:%M:%S %p')

IP addresses in the dataset are also formatted as strings, in a format like: ```192.168.0.1``` We use one of the built-in IPv4 primitive functions `ip2int()` to convert IPv4 addresses to their integer (int64) representations.

In [ ]:
%%time
gdf['src_ip'] = gdf['Src IP'].str.ip2int()
gdf['dst_ip'] = gdf['Dst IP'].str.ip2int()

# Data Exploration

## Dataset Summary

There's 89 features for each flow! You can find descriptions of all of them here: https://www.unb.ca/cic/datasets/ids-2018.html. Or look at the table below with some of the ones we'll be using highlighted below.


|Field | Description|
|---|---|
|Src IP | Source IP
|Src Port | Source Port
|Dst IP| Destination IP
|Dst Port| Destination Port
|Tot Fwd Pkts| Total Forward Packets
|Tot Bwd Pkts| Total Backward Packets
|Fwd Header Len| Forward Header Length
|Bwd Header Len| Backward Header Lenght
|Fwd Pkts/s| Foward Packets per second
|Bwd Pkts/s | Backward Packets per second

## Data Exploration

### Dataset Size and Data Types

We first get a sense of how large the dataset is, and what some column names and their associated data types are.

In [ ]:
print(gdf.shape)

In [ ]:
print(gdf.dtypes)

### Summary Statistics on Numeric Fields

Often it's useful to generate summary statistics on numeric fields. This is easy with the `describe()` function. Here, the output includes the minimum, maximum, mean, median, standard deviation, and various quantiles for selected fields in the dataset.

In [ ]:
print(gdf[['Flow Duration','Tot Fwd Pkts','Tot Bwd Pkts', 'Fwd Header Len', 'Bwd Header Len', 'Fwd Pkts/s', 
           'Bwd Pkts/s']].describe())

### Value Counts of Fields

Ports are interesting. They help us determine what applications are being used by referring back to [IANA service mappings](https://www.iana.org/assignments/service-names-port-numbers/service-names-port-numbers.xhtml). For example, port 53 (UDP protocol) is commonly associated with DNS, while ports 80, 443, and 8080 (TCP protocol) are associated with HTTP and HTTPS. For TCP and UDP, ports are interpreted as you would imagine. However, network data often includes [ICMP](https://en.wikipedia.org/wiki/Internet_Control_Message_Protocol) traffic, which has no traditional notion of ports. In these cases, values for ports are interpreted as the [ICMP type](https://en.wikipedia.org/wiki/Internet_Control_Message_Protocol#Control_messages).

Let's investigate some of the destination ports and their value counts in the dataset.

In [ ]:
# returns values from largest to smallest count
print(gdf['Dst Port'].value_counts())

If we sort by the port, we can see that port 0 is represented quite frquently. This typically corresponds with ICMP traffic, meaning an echo reply (used to ping assets on the network).

In [ ]:
# sort by port number
print(gdf['Dst Port'].value_counts().sort_index())

### Aggregate Statistics

Another common way to investigate a new dataset is to calculate aggregate statistics on various fields (and groupings of fields) in the data. Below, we apply a `groupby()` then a `sum()` to calculate total forward-direction (outbound) packets for each source IP address.

In [ ]:
print(gdf[['Src IP','Tot Fwd Pkts']].groupby('Src IP').sum())

## <font color="green">Code Me</font>

Use [cuDF functionality](https://rapidsai.github.io/projects/cudf/en/latest/index.html) to perform exploratory data analysis on additional fields.

In [ ]:
# your code here

## Visualizing Data Distribution

A big challenge for cyber security is to distinguish normal network traffic from malicious traffic. We can plot a feature's distribution to try to decide where a cut-offs for "normal" traffic could be.  

Let's start with something easy, ports (specifically destination ports, for the reasons above). 

In [ ]:
plotdf = cudf.DataFrame()
plotdf['Dst Port'] = gdf['Dst Port'].value_counts().head(30).sort_index().index
plotdf['Freq'] = gdf['Dst Port'].value_counts().head(30).sort_index()

In [ ]:
%%time
ports = plotdf['Dst Port']
y_pos = np.arange(len(ports))
frequency = plotdf['Freq']

plt.figure(figsize=(20,5))
plt.bar(y_pos, frequency, align='center', alpha=0.5)
plt.xticks(y_pos, ports)
plt.ylabel('frequency')
plt.xlabel('dest port')
plt.title('Top 30 Most Frequently Used Destination Ports')

plt.show()

Should we block traffic that uses abnormal ports? If only 0.00001% of traffic is malicious, we would have a very high-false positive rate and lots of unhappy users if we blocked the traffic using abnormal destination ports.

Let's look at a continuous field instead of a categorical one, `Total Forward Packets`.

In [ ]:
# plot a histogram of a sample of our dataset with a kde kernal density estimate overlaid 
sns.distplot(gdf['Tot Fwd Pkts'].head(1000))

It looks like our data has a long tail distribution. We can visualize it another way with a cummulative frequency histogram.

In [ ]:
# evaluate the histogram
values, base = np.histogram(gdf['Tot Fwd Pkts'].head(1000), bins=40)

#evaluate the cumulative
cumulative = np.cumsum(values)

# plot the cumulative function
plt.plot(base[:-1], cumulative, c='blue')
plt.show()

It looks like over 99.5% of our traffic has less than 25 total forward packets. Could we just flag all the traffic with greater than 25 forward packets as anaomolous? Is this a good idea? Why or why not?

Let's plot another data field, `Flow Duration`.

In [ ]:
#plot a histogram of a sample of our dataset with a kde kernal density estimate overlaid 
sns.distplot(gdf['Flow Duration'].head(1000))

It looks like `Flow Duration` is bimodal, what would it mean to have a single threshold to determine outliers within this field?

## <font color="green">Code Me</font>

Use another visual analysis to investigate data distributions. [Violin plots](https://seaborn.pydata.org/generated/seaborn.violinplot.html#seaborn.violinplot) could be informative.  

In [ ]:
# your code here

# Graph Representation of the Network Data

Networks (including cybersecurity networks) are frequently interpreted and represented as graphs. A graph representation affords us many benefits during analysis, including using both the structure, edge features, and generated features for anomaly detection. We first demonstrate how to create a [cuGraph](https://github.com/rapidsai/cugraph) representation of graph data represented in cuDF, then we walk through some analysis.

In [ ]:
G = cugraph.Graph()

In [ ]:
%%time
gdf['src_ip_r'], gdf['dst_ip_r'], numbering = cugraph.renumber(gdf['src_ip'], gdf['dst_ip'])

In [ ]:
gdf.head().to_pandas()

In [ ]:
%%time
G.add_edge_list(gdf['src_ip_r'],gdf['dst_ip_r'],None)

### Calculating the Degree

We'll find the number of connections at each node. This is often useful to see what nodes have the most connections, as these are typically backbone assets of the network.

In [ ]:
%%time
deg = G.degree()

In [ ]:
# top 3 most connected vertices 
print(deg.sort_values('degree', ascending=False).head(3))

This isn't super useful though since it's the internal cuGraph renumbering representation of the vertex. Ideally, we want the actual IPv4 address. We can do that via a quick [`merge` operation](https://rapidsai.github.io/projects/cudf/en/latest/api.html?highlight=merge#cudf.dataframe.DataFrame.merge).

In [ ]:
%%time
ip_lookup_gdf = cudf.concat([gdf[['Src IP','src_ip_r']].rename({'Src IP': 'ip','src_ip_r':'vertex'}),
                             gdf[['Dst IP','dst_ip_r']].rename({'Dst IP': 'ip','dst_ip_r':'vertex'})]) \
                .drop_duplicates()

In [ ]:
%%time
ip_to_vertex_gdf = deg.drop_duplicates() \
                        .merge(ip_lookup_gdf, how='left', left_on='vertex', right_on='vertex')

In [ ]:
# top 10 most connected vertices
ip_to_vertex_gdf.sort_values('degree', ascending=False).head(10).to_pandas()

Now we have a better idea of what IPs have the highest degree. It looks like `172.31.0.2` is the highest, followed by the `169.254.169.254` IP address and the `18.219.211.138` address. Unfortunately, only one of these (the last one) is responsible for launching any attacks in this dataset. How can we write a different analytic that will move that IP address to the top of our triage list?

## Say Hello to PageRank

PageRank (PR) is a [fairly well-known algorithm](https://en.wikipedia.org/wiki/PageRank), originally developed to rank web pages in Google search results. Traditionally, the PageRank algorithm outputs a probability distribution which represents the likelihood that a person randomly clicking on links will arrive at any particular page. We can use that same property to rank states of an attack graph.

Fortunately for us, cuGraph makes it easy to calculate the PR of each node in a graph.

In [ ]:
%%time
# Call cugraph.pagerank to get the pagerank scores
gdf_pr = cugraph.pagerank(G)

And here are some of the PR values.

In [ ]:
gdf_pr.head().to_pandas()

In order to find the most important node, we first find the maximum PR value

In [ ]:
pr_max = gdf_pr['pagerank'].max()
print(pr_max)

And we then write a function that can search for that vertex and return it.

In [ ]:
# Use this value in a query to find the vertex and convert back to IP
def print_pagerank_threshold(_df, t=0) :
    filtered = _df.query('pagerank >= @t')    
    for i in range(len(filtered)):
        print("The most important vertex is " + str(filtered['vertex'][i]) + 
            " with a pagerank score of " + str(filtered['pagerank'][i]))

In [ ]:
print_pagerank_threshold(gdf_pr, pr_max)

## <font color="green">Code Me</font>

Using what you know about cuDF and cuGraph, is there a different way you could write the function `print_pagerank_threshold`? What about if we want the actual IPv4 address and not the renumbered vertex ID?

In [ ]:
# your code here

We know either by looking or by writing some more code that the node with the maximum PR value corresponds to the IP address `172.31.0.2`. That's the IP address that had the highest degree, and it's not one of the IPs that responsible for the botnet attack. Well that wasn't super useful.

## Personalized PageRank to the Rescue

The issue for us is that PageRank sets the initial weights of all the nodes the same. In other words, it assumes a uniform starting probability for every node. What if we have _a priori_ information about the nodes? In this context, what if we know some of the victim IP addresses, and we're not sure we have identified all of them? We can use personalized PageRank (PPR) to assist.

We first construct a list of `training_victim_ips`. These are the IP addresses that we _know_ to be compromised. But since we have ground truth data, we also know _all_ of the affected IP addresses. We'll make another list for those (but we don't use them in training).

In [ ]:
# the victim IPs we're going to consider for training
training_victim_ips = ['172.31.69.23','172.31.69.17', '172.31.69.14', '172.31.69.12', '172.31.69.10', 
              '172.31.69.26', '172.31.69.29', '172.31.69.30']

# all of the victim IPs we know via ground truth
all_victim_ips = ['172.31.69.23','172.31.69.17', '172.31.69.14', '172.31.69.12', '172.31.69.10', '172.31.69.8', 
              '172.31.69.6', '172.31.69.26', '172.31.69.29', '172.31.69.30']

# we know the times of the attack, so we put the [start, stop] times here for reference
attack_time_1 = [1519999860, 1520004840]
attack_time_2 = [1520015040, 1520020500]

We can now filter out the attack flows. There were two attacks at two different times on this date. We construct two DFs for each attack. We then assign a label of `1` if the flow represents an attack and `0` if it's benign. We do this for all IP addresses in our training set over the two affected time periods.

TL;DR - we're constructing a labeled training set. More accurately in this case, we're constructing a dataset to help us set initial probabilities (weights) for the PPR algorithm.

In [ ]:
%%time
attack1_gdf = gdf[(gdf['Src IP'] == "18.219.211.138") & ((gdf['Dst IP'] == training_victim_ips[0]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[1]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[2]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[3]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[4]) |
                                                         (gdf['Dst IP'] == training_victim_ips[5]) |
                                                         (gdf['Dst IP'] == training_victim_ips[6]) |
                                                         (gdf['Dst IP'] == training_victim_ips[7])) &
                    (gdf['datetime'] > attack_time_1[0]) * (gdf['datetime'] < attack_time_1[1])]

attack2_gdf = gdf[(gdf['Src IP'] == "18.219.211.138") & ((gdf['Dst IP'] == training_victim_ips[0]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[1]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[2]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[3]) | 
                                                         (gdf['Dst IP'] == training_victim_ips[4]) |
                                                         (gdf['Dst IP'] == training_victim_ips[5]) |
                                                         (gdf['Dst IP'] == training_victim_ips[6]) |
                                                         (gdf['Dst IP'] == training_victim_ips[7])) &
                    (gdf['datetime'] > attack_time_2[0]) * (gdf['datetime'] < attack_time_2[1])]

attack_flow_ids_gdf = cudf.concat([attack1_gdf,attack2_gdf])['Flow ID'].unique().to_frame(name='Flow ID')
attack_flow_ids_gdf['Label'] = int(1)

gdf = gdf.merge(attack_flow_ids_gdf, left_on=['Flow ID'], right_on=['Flow ID'], how='left', suffixes=('_orig',''))
gdf['Label'] = gdf['Label'].fillna(0)

Just for fun, the cell below performs the same task using pandas. It's commented out, but feel free to uncomment and compare timings.

In [ ]:
# %%time
# gdf_as_pdf = gdf.to_pandas()

# attack1 = gdf_as_pdf[(gdf_as_pdf['Src IP'] == "18.219.211.138") & (gdf_as_pdf['Dst IP'].isin(training_victim_ips)) & 
#                    (gdf_as_pdf['datetime'] > attack_time_1[0]) & (gdf_as_pdf['datetime'] < attack_time_1[1])]

# attack2 = gdf_as_pdf[(gdf_as_pdf['Src IP'] == "18.219.211.138") & (gdf_as_pdf['Dst IP'].isin(training_victim_ips)) & 
#                    (gdf_as_pdf['datetime'] > attack_time_2[0]) & (gdf_as_pdf['datetime'] < attack_time_2[1])]

# attack_flow_ids = set(attack1['Flow ID']) | set(attack2['Flow ID'])

# gdf_as_pdf["Label"] = [int(1) if fid in attack_flow_ids else int(0) for fid in gdf_as_pdf['Flow ID']]

# gdf = cudf.DataFrame.from_pandas(gdf_as_pdf)

Let's inspect the head of the DF to make sure we have what we need.

In [ ]:
gdf[['Label', 'datetime','Src IP', 'Dst IP', 'src_ip_r', 'dst_ip_r']].head(10).to_pandas()

Let's think about what we have here. We have _known attacks_ on _known IPs_. We also know there are some unknown number of other IPs that could be impacted. We want a way to prioritize what other IPs to investigate. Personalized PageRank take a `personalization vector` as an initialization parameter. Technically, this vector contains the transport probabilities for each node in the vector. For us, this amounts to setting some initial weights that are not uniform.

But how to do this? One straightforward way is, for each node, to count the number of attack-labeled edges it has. This is a task made for a `groupby().count()`.

In [ ]:
%%time 
# Get only rows that are labeld as 1 (attack)
attacked_gdf = gdf[gdf['Label'] == 1]

# For each node, count how many attack-labeled edges it makes
aggregated_gdf = attacked_gdf.groupby(['src_ip_r']).count()

# Get personalization vector
# (make sure that the type of vertex is int32 and that of values is float32)
personalization_vec = cudf.DataFrame()
personalization_vec['vertex'] = aggregated_gdf.index.values
personalization_vec['values'] = aggregated_gdf['Timestamp'].astype('float32')

We now have everything we need to run PPR. In cuGraph, PPR is the same function as regular PR. The only change is that we add a `personalization` parameter and feed it in the personalization vector we computed above.

In [ ]:
%%time
# Run PPR
ppr = cugraph.pagerank(G, alpha=0.85, personalization=personalization_vec, max_iter=100, tol=1.0e-5, nstart=None)

Now what if we query the PPR results for the node with the maximum PPR value?

In [ ]:
ppr_max = ppr['pagerank'].max()

print_pagerank_threshold(ppr, ppr_max)

Looking back in our original dataset, we see that this vertex corresponds to the IP address `18.219.211.138`. This is the same IP address that is responsible for initiating the botnet attacks.

In [ ]:
gdf.query('src_ip_r == 11418').head(10).to_pandas()

Let's merge our PPR values back with this larger cuDF.

In [ ]:
%%time
gdf = ppr.merge(gdf, how='left', left_on='vertex', right_on='src_ip_r')

Now the moment of truth. If we sort our entire cuDF by the PPR value, we should find those with a maximum PPR at the top. The hypothesis is that these IPs are involved in the attacks.

In [ ]:
%%time 
gdf[['Src IP','Src Port','Dst IP','pagerank']] \
        .drop_duplicates() \
        .sort_values(by='pagerank', ascending=False) \
        .head(15) \
        .to_pandas()

In [ ]:
training_victim_ips

A visual inspection of this table shows us that there are IPs we suspect. Looking in the `Dst IP` column, we find `172.31.69.10`, `172.31.69.17`, `172.31.69.29`, and `172.31.69.30`. But we also see `172.31.69.6` and `172.31.69.8`. These were not in our original training dataset (meaning we assumed we didn't know about them), but they _are_ victims of the attack as shown in the ground truth dataset. Neat.

## <font color="green">Code Me</font>

We used simple counts to provide the personalization vector for PPR. There are other ways you could construct this vector. What about degree (especially degree in a digraph)?

In [ ]:
# your code here

## <font color="green">Code Me</font>

We've talked about PageRank and Personalized PageRank, but there are other graph analyitcs availabe in cuGraph. One of them is [Louvain modularity](https://rapidsai.github.io/projects/cugraph/en/0.8.0/api.html?highlight=louvain#cugraph.nvLouvain) (community detection). Calculating the partitions is fairly straightforward:

```louvain_parts, modularity_score = cugraph.louvain(G)```

Use Louvain modularity to partition the network graph, and investigate the results. Are all the the victim IPs in the same community? Would you expect them to be?

In [ ]:
# your code here

## Visualization

Many times, analysts are content (and prefer) raw analytic results that are enriched back to the original dataset and/or the [SIEM](https://searchsecurity.techtarget.com/definition/security-information-and-event-management-SIEM). But what if you do want to visualize some of these results? Here we show how some of that is possible, including a live demo using [Graphistry](https://www.graphistry.com).

![Visualization](./images/viz_1.jpg)

<hr />

## Acknowledgmnets

We would like to thank the [Canadian Institute for Cybersecurity](https://www.unb.ca/cic/) for the data used in this tutorial. A complete description of the dataset used is [available online](https://registry.opendata.aws/cse-cic-ids2018/). In addition, the paper associated with this dataset is:

> Iman Sharafaldin, Arash Habibi Lashkari, and Ali A. Ghorbani, “Toward Generating a New Intrusion Detection Dataset and Intrusion Traffic Characterization”, 4th International Conference on Information Systems Security and Privacy (ICISSP), Portugal, January 2018

We would also like to acknowledge the contributions of Eli Fajardo (NVIDIA), Brad Rees, PhD (NVIDIA), and the [RAPIDS](https://rapids.ai) engineering team.